In [57]:
import pandas as pd
import os
import json
import time
from tqdm import tqdm
from dotenv import load_dotenv
from openai import AzureOpenAI

In [58]:
load_dotenv()
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION="2023-05-15"

In [59]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version="2024-08-01-preview",
)

In [60]:
def get_user_prompt(text,genres,movie_data):
    
    #Function to Generate the Full User Prompt
    
    prompt = f'''
    This is a input provided by a user related to a movie genre.
    
    **Input Text**
    "{text}"

    This input is related to a movie genre. Your task is to analyze it and provide a response in the following JSON format:
    
    ```json
    {{
        "Movies": "One or more movie genre that best fits the user input(Limit it to a maximum of 5 movies)",
        "Genre": "List of comma separated genres that best fit the user input"
    }}
    ```
    
    **The different movie genres could be one from the following list:**
    - {genres}
    
    **The different list of movies and their respective genre are:**
    - {movie_data}

    '''

    return prompt

In [61]:
def model_response(sys_prompt,user_prompt):
    
    #Function to Generate  User Response using OPENAI
    
    response = client.chat.completions.create(
    model="gpt-35-turbo-16k",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ])
    return json.loads(response.choices[0].message.content)

In [62]:
def refresh_conversation(chat):
    
    # Function to Provide User Response
    
    movies_df = pd.read_csv("movies.csv")
    movies_df.head(10)
    genres= list()
    for i in movies_df['genres']:
        for j in i.split('|'):
            if j not in genres:
                genres.append(j)
    with open("movies.csv") as file:
        movie_data = file.read()
        
    sys_prompt = '''
    You are a helpful movie recommendation assistant that helps in identifying the genre of movies that best fits the user input, 
    and suggest one or more movies that the user would enjoy watching.
    '''    
    user_prompt = get_user_prompt(chat,genres,movie_data)
    response = model_response(sys_prompt,user_prompt)
    movies=response['Movies']
    genres_movie=response['Genre']
    print(f"Movie Recommendations : {movies} \nGenre of Interest: {genres_movie}")

In [63]:
def main():
    while True:
        #Take in User Input
        message = input("Please enter a message (or 'exit' to leave): ")
        # closing the chatbot.
        if message.lower() == 'exit':
            break  
        # Continue the conversation with the new message(if the message is not exit)
        refresh_conversation(message)


if __name__ == '__main__':
    main()

Movie Recommendations : Guardians of the Galaxy, Star Trek: Into Darkness, Deadpool 2 
Genre of Interest: Action, Comedy, Sci-Fi
